#### Evaluating 3DMC

In [1]:
from GAME.bin.intertask_mappings import *
from GAME.utils.config import config
import os
import numpy as np

In [2]:
config_data = config()

MC2D_states = ['x_position', 'x_velocity']
MC3D_states = ['x_position', 'x_velocity', 'y_position', 'y_velocity']
MC2D_actions = ['Left', 'Neutral', 'Right']
MC3D_actions = [0, 1, 2, 3, 4]

# evolution parameters
src_state_var = MC2D_states
src_actions = MC2D_actions
target_state_var = MC3D_states
target_action_values = MC3D_actions

src_data_path = os.path.join(config_data['data_path'], 'mountain_car', "MC2D_transitions.csv")
src_data_path = os.path.join(config_data['output_path'], '12142022 2DMC Sample Collection 200 Episodes with Training', '2DMC_100_episodes_sample_data_small.csv')
src_data_df = pd.read_csv(src_data_path, index_col = False)
transformed_df_current_state_cols = config_data['3DMC_current_state_transition_df_col_names']
transformed_df_next_state_cols = config_data['3DMC_next_state_transition_df_col_names']
transformed_df_col_names = config_data['3DMC_full_transition_df_col_names']

# network_folder_path = os.path.join(config_data['pickle_path'], 'neural_nets', "mountain_car")
network_folder_path = os.path.join(config_data['pickle_path'], "12142022 MC3D Neural Nets")
network_folder_path = os.path.join(config_data['pickle_path'], "01072023 3DMC Transition Approx MSE")
eval_networks = EvaluationNetworks(network_folder_path)

In [3]:
len(src_data_df)

60000

In [97]:
num_min_samples = src_data_df[src_data_df["Current-action"] == 1].count()[0]
num_min_samples = 20000

src_df_left = src_data_df[src_data_df["Current-action"] == 0]
remove_n = src_df_left.count()[0] - num_min_samples
drop_indices_left = np.random.choice(src_df_left.index, remove_n, replace=False)
src_df_right = src_data_df[src_data_df["Current-action"] == 2]
remove_n = src_df_right.count()[0] - num_min_samples
drop_indices_right = np.random.choice(src_df_right.index, remove_n, replace=False)
src_df_neutral = src_data_df[src_data_df["Current-action"] == 1]
remove_n = src_df_neutral.count()[0] - num_min_samples
drop_indices_neutral = np.random.choice(src_df_neutral.index, remove_n, replace=False)
src_data_df = src_data_df.drop(list(drop_indices_left) + list(drop_indices_right) + list(drop_indices_neutral))
src_data_df = src_data_df.reset_index(level = 0)

In [98]:
len(src_data_df)

60000

In [102]:
src_data_df.to_csv(os.path.join(config_data['output_path'], '12142022 2DMC Sample Collection 200 Episodes with Training', '2DMC_100_episodes_sample_data_small.csv'), index=False)

In [4]:
mapping = IntertaskMapping([0, 1, 0, 1], [1, 0, 2, 0, 2], src_state_var, src_actions, target_state_var, target_action_values)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [84]:
transformed_df

,Current-x_position,Current-x_velocity,Current-y_position,Current-y_velocity,Next-x_position,Next-x_velocity,Next-y_position,Next-y_velocity,Current-action-0,Current-action-1,Current-action-2,Current-action-3,Current-action-4
0,-0.445959,-0.001168,-0.445959,-0.001168,-0.447705,-0.001745,-0.447705,-0.001745,1.0,0.0,0.0,0.0,0.0
1,-0.447705,-0.001745,-0.447705,-0.001745,-0.450014,-0.002309,-0.450014,-0.002309,1.0,0.0,0.0,0.0,0.0
2,-0.450014,-0.002309,-0.450014,-0.002309,-0.452871,-0.002857,-0.452871,-0.002857,1.0,0.0,0.0,0.0,0.0
3,-0.452871,-0.002857,-0.452871,-0.002857,-0.456254,-0.003383,-0.456254,-0.003383,1.0,0.0,0.0,0.0,0.0
4,-0.456254,-0.003383,-0.456254,-0.003383,-0.461139,-0.004885,-0.461139,-0.004885,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14821,0.399294,0.018069,0.399294,0.018069,0.417452,0.018159,0.417452,0.018159,0.0,0.0,1.0,0.0,1.0
14822,0.417452,0.018159,0.417452,0.018159,0.435828,0.018376,0.435828,0.018376,0.0,0.0,1.0,0.0,1.0
14823,0.435828,0.018376,0.435828,0.018376,0.454553,0.018725,0.454553,0.018725,0.0,0.0,1.0,0.0,1.0
14824,0.473764,0.019211,0.473764,0.019211,0.493603,0.019839,0.493603,0.019839,0.0,0.0,1.0,0.0,1.0


In [5]:
mapping = IntertaskMapping([0, 1, 0, 1], [1, 0, 2, 0, 2], src_state_var, src_actions, target_state_var, target_action_values)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

print(parse_mapping_eval_scores(eval_results))

[0.9999445868986845]


In [100]:
mapping = IntertaskMapping([0, 1, 0, 1], [1, 0, 2, 0, 2], src_state_var, src_actions, target_state_var, target_action_values)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

print(parse_mapping_eval_scores(eval_results))

[0.9999445868986617]


In [101]:
mapping = IntertaskMapping([0, 1, 0, 1], [0, 0, 0, 0, 0], src_state_var, src_actions, target_state_var, target_action_values)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

print(parse_mapping_eval_scores(eval_results))

[0.999943681963089]


In [81]:
transformed_df.head()

,Current-x_position,Current-x_velocity,Current-y_position,Current-y_velocity,Next-x_position,Next-x_velocity,Next-y_position,Next-y_velocity,Current-action-0,Current-action-1,Current-action-2,Current-action-3,Current-action-4
0,-0.445959,-0.001168,-0.445959,-0.001168,-0.447705,-0.001745,-0.447705,-0.001745,0.0,0.0,0.0,0.0,0.0
1,-0.447705,-0.001745,-0.447705,-0.001745,-0.450014,-0.002309,-0.450014,-0.002309,0.0,0.0,0.0,0.0,0.0
2,-0.450014,-0.002309,-0.450014,-0.002309,-0.452871,-0.002857,-0.452871,-0.002857,0.0,0.0,0.0,0.0,0.0
3,-0.452871,-0.002857,-0.452871,-0.002857,-0.456254,-0.003383,-0.456254,-0.003383,0.0,0.0,0.0,0.0,0.0
4,-0.456254,-0.003383,-0.456254,-0.003383,-0.461139,-0.004885,-0.461139,-0.004885,1.0,1.0,1.0,1.0,1.0


In [22]:
print(parse_mapping_eval_scores(eval_results))

[0.999632018778979]


In [99]:
print(parse_mapping_eval_scores(eval_results))

[0.8208388944232995]


In [64]:
eval_results

{'0--Next-x_position': 0.9996365345062659,
 '0--Next-x_velocity': 0.9999147884052522,
 '0--Next-y_position': 0.9999579586607795,
 '0--Next-y_velocity': 0.9999958375860888,
 '1--Next-x_position': 0.9993169961671527,
 '1--Next-x_velocity': 0.9997771319867677,
 '1--Next-y_position': 0.9989816803527768,
 '1--Next-y_velocity': 0.9999913428013781,
 '2--Next-x_position': 0.9996182054775111,
 '2--Next-x_velocity': 0.9999281911459168,
 '2--Next-y_position': 0.999968632559012,
 '2--Next-y_velocity': 0.9998100674685203,
 '3--Next-x_position': 0.9989166044709156,
 '3--Next-x_velocity': 0.9999885145091619,
 '3--Next-y_position': 0.9989099008996224,
 '3--Next-y_velocity': 0.9999908372720023,
 '4--Next-x_position': 0.998926018113497,
 '4--Next-x_velocity': 0.9999916533530876,
 '4--Next-y_position': 0.9999278873332571,
 '4--Next-y_velocity': 0.9999893851788749}

#### Evaluating 4v3 mappings

In [1]:
from GAME.bin.intertask_mappings import *
from GAME.utils.config import config
from itertools import product
from sklearn.preprocessing import MinMaxScaler

In [2]:
def agg_res(eval_results:dict, state:str=None, action:int=None, strategy:str='average') -> dict:
    agg_results = {}
    state_agg = []
    action_agg = []
    for k, v in eval_results.items():
        key_split = k.split('--')
        key_state = key_split[1].split('-')[1]
        key_action = key_split[0]
        if state and state == key_state:
            state_agg.append(v)
        if action and str(action) == key_action:
            action_agg.append(v)
    if strategy == 'average':
        if state:
            agg_results[state] = np.mean(state_agg)
        if action:
            agg_results[action] = np.mean(action_agg)
    if strategy == 'best':
        if state:
            agg_results[state] = np.max(state_agg)
        if action:
            agg_results[action] = np.max(action_agg)

    return agg_results

In [21]:
# load config data
config_data = config()

src_state_var_names = config_data['3v2_state_names']
src_action_names = config_data['3v2_action_names']
src_action_values = config_data['3v2_action_values']
target_state_var_names = config_data['4v3_state_names']
target_action_names = config_data['4v3_action_names']
target_action_values = config_data['4v3_action_values']

# mapping = IntertaskMapping([0 for _ in range(19)], [0, 1, 2, 2], src_state_var_names, src_action_names, target_state_var_names, target_action_names)
# mapping = IntertaskMapping([11, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12], [0, 1, 2, 2], src_state_var_names, src_action_names, target_state_var_names, target_action_names)

src_data_path = os.path.join(config_data['data_path'], 'keepaway', "keepaway_3v2_transitions.csv")
src_data_df = pd.read_csv(src_data_path, index_col = False)
transformed_df_col_names = config_data['4v3_full_transition_df_col_names']

feature_scaler = MinMaxScaler()
feature_names = ['Current-{}'.format(feature) for feature in src_state_var_names]
feature_scaler.fit(src_data_df[feature_names])
src_data_df[feature_names] = feature_scaler.transform(src_data_df[feature_names])

target_scaler = MinMaxScaler()
target_names = ['Next-{}'.format(feature) for feature in src_state_var_names]
target_scaler.fit(src_data_df[target_names])
src_data_df[target_names] = target_scaler.transform(src_data_df[target_names])

# transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)
# transformed_df_out_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\output\\11022022 Transformed Source Data 3DMC\\transformed_df.csv"
# transformed_df.to_csv(transformed_df_out_path, index = False)

network_folder_path = os.path.join(config_data['pickle_path'], 'neural_nets', "keepaway")
eval_networks = EvaluationNetworks(network_folder_path)
# test_mlp = eval_networks.get_network(0, 'Next_x_position')

transformed_df_current_state_cols = config_data['4v3_current_state_transition_df_col_names']
transformed_df_next_state_cols = config_data['4v3_next_state_transition_df_col_names']

# eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [ ]:
state_mapping = [0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12]
state_mapping = np.random.choice(13, size = 19)

for action_mapping in product(src_action_values, repeat = len(target_action_values)):
    mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

    transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

    eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

    print("Action mapping: {}, Results: {}".format(action_mapping, parse_mapping_eval_scores(eval_results)))

In [24]:
state_mapping = [0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12]
# state_mapping = [0 for _ in range(19)]
action_mapping = [0, 1, 2, 2]
mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [25]:
print(parse_mapping_eval_scores(eval_results))

[0.7288566720658226]


In [26]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.8722644839350171}
{'dist(K1,K2)': -0.22939281778175027}
{'dist(K1,K3)': 0.8424341522855624}
{'dist(K1,K4)': 0.9286357208327285}
{'dist(K1,T1)': 0.825480264121766}
{'dist(K1,T2)': 0.8231096644799776}
{'dist(K1,T3)': 0.8276272203865441}
{'dist(K2,C)': 0.6698307880330103}
{'dist(K3,C)': 0.7540449851072928}
{'dist(K4,C)': 0.8598177118728227}
{'dist(T1,C)': 0.922598368239199}
{'dist(T2,C)': 0.9282651407342403}
{'dist(T3,C)': 0.9304682075749454}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.7970593744195932}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.6786768532653569}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': -0.17427110914843588}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.9465057040889643}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.7738532579239163}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.8712687988798752}


In [28]:
# GAME
state_mapping = [0, 11, 5, 0, 4, 3, 4, 9, 6, 6, 7, 9, 9, 8, 2, 4, 11, 10, 9]
# state_mapping = [0 for _ in range(19)]
action_mapping = [0, 0, 0, 0]
mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [29]:
print(parse_mapping_eval_scores(eval_results))

[0.9094175953584726]


In [30]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.8984891033086441}
{'dist(K1,K2)': 0.9832637591162632}
{'dist(K1,K3)': 0.7849590820300703}
{'dist(K1,K4)': 0.954064586502867}
{'dist(K1,T1)': 0.8867525246875843}
{'dist(K1,T2)': 0.9106655510397274}
{'dist(K1,T3)': 0.9243453270534202}
{'dist(K2,C)': 0.8889423560018718}
{'dist(K3,C)': 0.9271890722436928}
{'dist(K4,C)': 0.9442787800076675}
{'dist(T1,C)': 0.8949127149320598}
{'dist(T2,C)': 0.9360098592514783}
{'dist(T3,C)': 0.9502627358934586}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.739350172055359}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.8737771738760037}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': 0.9439320299226585}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.9525089079653637}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.9239104302786245}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.9613201456441616}


In [13]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.49881693885453054}
{'dist(K1,K2)': -0.3500011293129991}
{'dist(K1,K3)': 0.4865951172581645}
{'dist(K1,K4)': 0.5711864998094486}
{'dist(K1,T1)': 0.412767853771606}
{'dist(K1,T2)': 0.4277775505462874}
{'dist(K1,T3)': 0.3925239806523952}
{'dist(K2,C)': 0.6575082449014189}
{'dist(K3,C)': 0.488432748544326}
{'dist(K4,C)': 0.6371720118516084}
{'dist(T1,C)': 0.4963605488009031}
{'dist(T2,C)': 0.49763661988783015}
{'dist(T3,C)': 0.48662396797778407}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.5987640349561779}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.6503778129774447}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': 0.2190113116089851}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.8303647337685836}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.536035884773041}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.7295451004304401}


In [14]:
for action in target_action_values:
    print(agg_res(eval_results, None, str(action), 'average'))

{'0': 0.5970050794532115}
{'1': 0.46486781561206886}
{'2': 0.4557285154566266}
{'3': 0.4334511857008248}


In [16]:
for action in target_action_values:
    print(agg_res(eval_results, None, str(action), 'average'))

{'0': 0.5970050794532115}
{'1': 0.6328943303695123}
{'2': 0.6278508153150593}
{'3': 0.5919189748063395}


In [ ]:
state = 'dist(K1,C)'
action = '0'

In [7]:
for k, v in eval_results.items():
    print(k, v)

0--Next-dist(K1,C) 0.5565052148801035
0--Next-dist(K1,K2) 0.9233535665106078
0--Next-dist(K1,K3) 0.9649852551019529
0--Next-dist(K1,K4) 0.853310103439609
0--Next-dist(K1,T1) 0.6395998534270293
0--Next-dist(K1,T2) 0.8772711177089803
0--Next-dist(K1,T3) 0.8463707785670385
0--Next-dist(K2,C) 0.9146010356947873
0--Next-dist(K3,C) 0.8934455980799146
0--Next-dist(K4,C) -9.121264605889044
0--Next-dist(T1,C) 0.19186391483142595
0--Next-dist(T2,C) 0.8462037608124144
0--Next-dist(T3,C) 0.7670662733368777
0--Next-Min(dist(K2,T1),dist(K2,T2),dist(K2,T3)) 0.8671977978044136
0--Next-Min(dist(K3,T1),dist(K3,T2),dist(K3,T3)) 0.7494318067170905
0--Next-Min(dist(K4,T1),dist(K4,T2),dist(K4,T3)) 0.7978817797390454
0--Next-Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3)) 0.2854170578096962
0--Next-Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3)) 0.15549082839276507
0--Next-Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3)) 0.6990632346124325
1--Next-dist(K1,C) 0.9387862035367304
1--Next-dist(K1,K2) -7.412251696456